In [1]:
!pip install matplotlib keras tensorflow transformers gpt_2_simple openai
%pylab inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 35.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 5.1 MB/s eta 0:00:00
  Created wheel for gpt_2_simple: filename=gpt_2_simple-0.8.1-py3-none-any.whl size=24576 sha256=c29b8ca66848a094552c87907ff5427227df59df1820245f980a7cc0a9aa5808
  Stored in directory: /root/.cache/pip/wheels/4e/28/f0/2f

In [2]:
import tensorflow as tf
import os, openai, re, pandas as pd
from functools import reduce

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
os.getcwd(), os.listdir(os.path.join(os.getcwd()))
change_to_colab = "drive/MyDrive/Colab Notebooks/rate-features-generation/"
# chdir to content/drive/MyDrive/Colab Notebooks
os.chdir(change_to_colab)

**Load review data**

In [8]:
# load summary revuews - sentence level
summ_dir = ["hybrid-tfidf-summary"]
domains = ["ride", "investing", "health"]

print("current dir: ", os.getcwd(), os.listdir())

tfidf_summ_reviews = {}
for domain in domains:
    reviews = pd.read_csv(summ_dir[0] +  "/" + domain + "_summary.csv")
    tfidf_summ_reviews[domain] = reviews
tfidf_summ_reviews["ride"].head()

current dir:  /content/drive/MyDrive/Colab Notebooks/rate-features-generation ['hybrid-tfidf-summary']


,doc_id,score,raw,sent,lemmatized
0,27783,1.205823,i am afairlynew lyftriderand iheardheard that ...,i am afairlynew lyftriderand iheardheard that ...,"competition,lyft,find,true,ber,seem,much,well,..."
1,26214,1.180414,usually cheaper than uber not cheaper than a c...,usually cheaper than uber not cheaper than a c...,"usually,cheaper,uber,cheaper,cab,reliable,usua..."
2,28885,1.076704,well it's been a roller coaster riding with ly...,well it's been a roller coaster riding with ly...,"well,roller,coaster,rid,lyft,start,use,lyft,we..."
3,292,0.874847,better & cheaper than train/bus/taxi. some dri...,also will it would be nice if there was an opt...,"also,would,nice,option,change,time,offer,money..."
4,62785,0.802241,i hate via they take way to long they take you...,i hate via they take way to long they take you...,"hate,via,take,way,long,take,money,paid,via,pas..."


In [9]:
tfidf_summ_reviews["health"].head()

,doc_id,score,raw,sent,lemmatized
0,25004,0.782249,had to go back & forth with customer service t...,had to go back & forth with customer service t...,"back,forth,customer,service,get,price,two,frie..."
1,63148,0.705859,just downloaded rootd today & i think itz such...,just downloaded rootd today & i think itz such...,"download,rootd,today,think,cute,lil,app,help,m..."
2,40152,0.580661,this app is gawd awful!! i accidentally downlo...,i accidentally downloaded it(via my niece) and...,"accidentally,download,via,niece,get,update,say..."
3,26196,0.440951,this app is for anxiety and to help people who...,this app is for anxiety and to help people who...,"app,anxiety,help,people,trouble,fall,asleep,be..."
4,4375,0.400531,i've been using betterhelp for maybe six month...,i've been using betterhelp for maybe six month...,"use,betterhelp,maybe,six,month,student,discoun..."


In [10]:
tfidf_summ_reviews["investing"].head()

,doc_id,score,raw,sent,lemmatized
0,37683,0.806133,m1 vs webull hands down m1 is better then webu...,webull has a stock lending program allows you ...,"webull,stock,lending,program,allows,loan,share..."
1,33941,0.743765,i am a disabled american man that was excited ...,i am a disabled american man that was excited ...,"disabled,american,man,excite,opening,account,p..."
2,26125,0.624518,i gave one star because had to do something to...,i gave one star because had to do something to...,"give,one,star,something,proceed,download,app,p..."
3,26484,0.513329,nothing works on apple since march 2020 - not ...,an issue with broken watch lists on apple ipad...,"issue,broken,watch,list,apple,ipad,fidelity,ap..."
4,213744,0.456688,if i can give this no stars i wouldthis is the...,if i can give this no stars i wouldthis is the...,"give,star,bad,place,crypto,first,take,money,sa..."


In [37]:
import itertools

# https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api

openai.api_key = OPEN_AI_API

gpt_label_dir = "gpt_labels/"
summ_dir = ["hybrid-tfidf-summary"]
domains = ["ride", "investing", "health"]

def get_labels_from_summ_reviews(top_summ, params, output_file):
    result = []
    print("params: ", params)
    for summ in top_summ:
        # prompt = summ + "\n" + params["prefix"]
        prompt = params["prefix"] + "Review: \"\"\"\n"+ summ + "\n\"\"\"\""
        print("\nprompt:\n", prompt)
        response = openai.ChatCompletion.create(
                        model="gpt-3.5-turbo",
                        messages = [{"role": "user", "content": prompt}] ,
                        max_tokens=params["max_tokens"], 
                        n=1, # number of output from the prompt
                        stop= None,
                        top_p=params["top_p"],
                        frequency_penalty = params["frequency_penalty"],
                        presence_penalty = params["presence_penalty"],
                        temperature=params["temperature"])
        labels = response.choices[0].message.content.strip().split("\n")
        print("labels: ", labels)
        result.append((params, summ, prompt, response.choices, labels))
    
    df = pd.DataFrame(result, columns=["params", "input",  "prompt", "generator choices", "labels"])
    df.to_csv(output_file, index=False, header=True)
    print("\n-------------- saved result to ", output_file, "------------\n")

    
def get_params(domain):

    grid_params = {"temperature": [0.1],
            "top_p": [0.1],
            "max_tokens": [150], 
            "presence_penalty": [0.1], 
            "frequency_penalty": [0.1],
            "prefix": [
               "\nGenerate minimum of five labels (two to three words) with very concise descriptions that best summarize the following "  + domain + " app review.\nDesired format: {label}: {description}\n",
                "\nGenerate minimum of five adjective labels with very concise descriptions that best summarize the following "  + domain + " app review.\nDesired format: {label}: {description}\n",
                "\nGenerate minimum of five labels for Non Functional Requirements that best summarize the following " + domain + " app review.\nDesired format: {label}: {description}\n",
                "\nGenerate minimum of five labels for Non Functional Requirements with very concise descriptions that summarize the following " + domain + " app review.\nDesired format: {label}: {description}\n",
                ]
            }

    # Generate all possible combinations of parameters
    param_combinations = list(itertools.product(
        grid_params["temperature"], 
        grid_params["top_p"], 
        grid_params["max_tokens"], 
        grid_params["presence_penalty"], 
        grid_params["frequency_penalty"], 
        grid_params["prefix"]
    ))

    # Loop through parameter combinations and perform training and evaluation
    combinations = []
    for params in param_combinations:
        temperature, top_p, max_tokens, presence_penalty, frequency_penalty, prefix = params
        item = {"prefix": prefix, "top_p": top_p, 
            "temperature": temperature, "max_tokens": max_tokens,
            "presence_penalty": presence_penalty, "frequency_penalty": frequency_penalty}
        combinations.append(item)
    print("combinations: ", len(combinations))
    return combinations

gpt_embed_raw = {"ride": {}, "health": {}, "investing": {}}
gpt_embed_vector = {"ride": {}, "health": {}, "investing": {}}
domain_keys = {"ride": "ride-hailing", "health": "mental health", "investing": "investing"}

print("---------------- raw sentences --------------------")
num_reviews = 10
domains = ["ride", "investing", "health"]

for domain in domains:
    print("\n\n------------domain: ", domain, "------------------")
    params = get_params(domain_keys[domain])
    tmp = tfidf_summ_reviews[domain]
    reviews = tmp["sent"].tolist()[:num_reviews]
    for index, param in enumerate(params):
        output_file = gpt_label_dir + domain + "_" + str(index + 6) + "_gpt_labels.csv"
        print("output_file:" , output_file)
        get_labels_from_summ_reviews(reviews, param, output_file)

---------------- raw sentences --------------------


------------domain:  investing ------------------
combinations:  4
output_file: gpt_labels/investing_6_gpt_labels.csv
params:  {'prefix': '\nGenerate minimum of five labels (two to three words) with very concise descriptions that best summarize the following investing app review.\nDesired format: {label}: {description}\n', 'top_p': 0.1, 'temperature': 0.1, 'max_tokens': 150, 'presence_penalty': 0.1, 'frequency_penalty': 0.1}

prompt:
 
Generate minimum of five labels (two to three words) with very concise descriptions that best summarize the following investing app review.
Desired format: {label}: {description}
Review: """
webull has a stock lending program allows you to loan your shares to short sellers to pull in extra income - winner m1 should adopt m1 has a barrow they will lend you money based on your stock value and stock holdings useful for higher income people m1 vs public public hands down kicks m1 in the face on investing 